<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

!{sys.executable} -m pip install --force-reinstall --upgrade "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"
import craid.eddb.loader.DataProducer as dp
import pandas as pd
import logging
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import timedelta, date


logging.getLogger().addHandler(logging.StreamHandler())
logging.getLogger().level = logging.DEBUG

arrays = dp.getDataArrays()
baseFrame: pd.DataFrame = arrays['dataFrame']

  Cloning https://github.com/HausReport/ClubRaiders.git to /tmp/pip-install-8dip6f_b/craid
  Running command git clone -q https://github.com/HausReport/ClubRaiders.git /tmp/pip-install-8dip6f_b/craid
Processing /root/.cache/pip/wheels/8f/56/d1/def235a9ef977abf4476d1ad7a3ad8d9af7537445ce01c5e16/dash-1.16.3-cp36-none-any.whl
Processing /root/.cache/pip/wheels/29/11/a0/0cf8183b6f7df4716bec25b0e089043cd161bb58accc6633a6/dash_core_components-1.12.1-cp36-none-any.whl
Processing /root/.cache/pip/wheels/8d/01/46/78e4de185a8a4a2da8ba31da16c52170f036d4cebeeb6e07a2/dash_html_components-1.1.1-cp36-none-any.whl
Processing /root/.cache/pip/wheels/3d/c2/fd/50a7ea0b9aef63cf3bf3c5fb22799b8ba9a97ea8f9b14b7124/dash_table-4.10.1-cp36-none-any.whl
Processing /root/.cache/pip/wheels/c8/8f/0f/9cf71e7362b302350e10fcc9ca7f0377b84a6ea14445c2d6aa/dash_renderer-1.8.2-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/28/2a03252dfb9ebf377f40fba6a7841b47083260bf8bd8e737b0c6952df83f/Flask-1.

Acquired lock.
Loading from AWS
Memory tracker: 0:167,284,736
Found data file: /tmp/smol-factions.jsonl.gz
Read 626 lines of faction data
Memory tracker: 0.5:167,571,456
Found data file: /tmp/smol-systems_populated.jsonl.gz
Read 165 lines of systems data
Memory tracker: 1:170,377,216
Populated club system keys
Club system lookup set has {165} items
Populated faction instances: {991} items.
Memory tracker: 2:171,307,008
Found data file: /tmp/smol-stations.jsonl.gz
Read 540 lines of station data, adding 540
Memory tracker: 3:171,847,680
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controllin

In [2]:
#pip install "pandas-profiling"


In [3]:
from craid.eddb.loader.strategy.DataLoader import DataLoader
from craid.eddb.loader.strategy.GithubLoader import LoadDataFromGithub
from craid.history.History import History

myLoader = LoadDataFromGithub(_forceWebDownload=True, useSmol=False)
hist = History()
csa = hist.getRawDataFrame()
target = hist.getNormalizedDataFrame()

Found data file: /tmp/history.jsonl.gz
NumExpr defaulting to 2 threads.


Creating the object


Read system        17756
faction       17756
updated       17756
influence     17756
control       17756
region        17756
population    17756
dtype: int64 lines of history data


In [4]:
def getScroller(fig):
  fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
  ))

In [5]:
def getFactionInfluenceGraph(fac: str):
  ogun = target[ target['faction']==fac]
  a = ogun['system'].unique().tolist()
  fig = go.Figure()
  for sys in a:
    theSlice = ogun[ ogun['system'] == sys].reset_index()
    fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    stackgroup="b",
                    name=sys,
                    stackgaps='interpolate'))
  getScroller(fig)
  return fig

In [6]:
def getSystemInfluenceGraph(sys: str):
  ogun = target[ target['system']==sys]
  a = ogun['faction'].unique().tolist()
  fig = go.Figure()
  for fac in a:
    theSlice = ogun[ ogun['faction'] == fac].reset_index()
    fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    name=fac))
  getScroller(fig)
  return fig

In [7]:
# data cleaning
# done in api now
#csa = csa.drop_duplicates()
#csa = csa[ csa['faction'] != 'Aegis of Federal Democrats']
#csa = csa[ csa['faction'] != 'Aegis Imperium']
#csa = csa[ csa['faction'] != "Emperor's Dawn"]

In [8]:
stage1 = csa.loc[csa.groupby(['faction','system']).updated.idxmax()]
stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
fig = px.pie(stage3, values='influence', names='faction', title='Total Influence by Minor Faction', width=350, height=350)
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [9]:
from craid.club.regions.RegionFactory import RegionFactory
stage3 = stage1[['region','influence']].groupby('region').influence.sum().reset_index()
regionNames = list(RegionFactory.regionDict.keys())
fig = px.pie(stage3, values='influence', names=regionNames, title='Total Club Influence by Region', width=350, height=350) #, names='faction'
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [10]:
region = 1
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence <br>by Minor Faction <br>in the {regionName} region"
merope = target[ target['region']== region]
stage1 = merope.loc[merope.groupby(['faction','system']).updated.idxmax()]
stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
fig = px.pie(stage3, values='influence', names='faction', title=title, width=350, height=350) #, names='faction'
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [11]:
theDate = '2020-06-1'
#reg = csa[ csa['region']== 1]
dated = csa [ csa['updated']<= theDate]

stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
#stage1[['system','faction','influence']]
stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
stage3.influence.sum()

4268.610099040681

In [12]:
region = 8

dates = []
inf = []

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) 

dframe = csa[ csa['region'] == region]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Club Influence in the {regionName} region"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)
      
labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [13]:

dates = []
inf = []

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) 

dframe = csa
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Club Influence"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)

labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [14]:
dates = []
inf = []

faction = "Sirius Corporation"
faction = "Hodack Prison Colony"
faction = "Wreaken Construction"
#faction = "'Sirius Drives'"
faction = "Aegis Core"
#faction = "Aegis Research"
#faction = "Bill Turner"

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) #date(2020, 6, 28)


dframe = csa[ csa['faction'] == faction]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence of the {faction} Faction"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)


labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [15]:
getFactionInfluenceGraph("Aegis Core").show()

In [16]:
badGuys= ['Abroin Universal PLC',
                     'Aegis Core', 'Aegis Defense', 'Aegis Research',
                      'Bill Turner',
                     'CQC Holdings',
                     'Gallant Investment Brokers', 'Hodack Prison Colony',
                     'Janus Incorporated', "Namarii Emperor's Dawn",
                     'Pleiades Resource Enterprise',
                     'Reyan BPS', 'Reynhardt IntelliSys',
                     'Sirius Atmospherics', 'Sirius Catering', 'Sirius Corporation',
                     'Sirius Drives', 'Sirius Hot2Cold', 'Sirius Hyperspace',
                     'Sirius Industrial', 'Sirius Luxury Transports', 'Sirius Mining Merope',
                     'Sirius Mining', 'Sirius Power',
                     'The Greenventure Group',
                     'The Peterson Group', 'The Rockforth Corporation',
                     'Turner Research Group', 'Wiggins Development Trust',
                     'Wreaken Construction'
                     ]

In [17]:
a = csa['faction'].unique().tolist()
b = baseFrame['factionName'].unique().tolist()
aa = set(a)
bb = set(b)
bb-aa

set()

In [18]:
dates = []
inf = []

faction = "Combined Sirius Factions"
faction1 = "Sirius"

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) #date(2020, 6, 28)


dframe = target[ (target['faction'].str.contains(faction1)) ]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence of the {faction} Faction"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)


labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [19]:
# One System, handles Multiple Factions
fig = getSystemInfluenceGraph("Celaeno")
fig.show()

In [20]:
fig = getSystemInfluenceGraph("Solati")
fig.show()

In [21]:
# One System, handles Multiple Factions
fig = getSystemInfluenceGraph("Sol")
fig.show()

In [22]:
fac = "Hodack Prison Colony"
ogun = target[ target['faction']==fac]
a = ogun['system'].unique().tolist()
fig = go.Figure()
for sys in a:
  theSlice = ogun[ ogun['system'] == sys].reset_index()
  fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    name=sys,
                    stackgroup="one",
                    stackgaps="interpolate"))

getScroller(fig)
fig.show()


In [23]:
getFactionInfluenceGraph('Hodack Prison Colony').show()

In [24]:
fac = 'Sirius Mining'


fac = 'Aegis Core'
fac = 'Sirius Mining Merope'
fac = 'CQC Holdings'
getFactionInfluenceGraph(fac).show()

In [25]:
from typing import List
def getCombinedFactionGraph( factions: List[str], titleSubject=None):
  labels = {'x':'Day', 'y':"Total Influence"}
  fig = go.Figure()
  for fac in factions:
    foo = target[target['faction']== fac]
    stage3 = foo[['system','faction','influence','updated']].groupby(['updated']).influence.sum().reset_index()
    fig.add_trace(go.Scatter(x=stage3['updated'], y=stage3['influence'],
                    mode='lines',
                    stackgroup="b",
                    name=fac,
                    stackgaps="interpolate"))

  if titleSubject is None:
    titleSubject = "+".join(factions)

  title = f"Total influence of {titleSubject}"
  fig.update_layout(title=title,
                    xaxis_title="Date",
                    yaxis_title="Total Influence")
  getScroller(fig)
  return fig



In [26]:
factions = ["Wreaken Construction", "Hodack Prison Colony"]

fig = getCombinedFactionGraph( factions, titleSubject=None)
fig.show()

In [27]:
siriusFactions = ['Sirius Atmospherics', 'Sirius Catering',
       'Sirius Corporation', 'Sirius Drives', 'Sirius Hot2Cold',
       'Sirius Hyperspace', 'Sirius Industrial',
       'Sirius Luxury Transports', 'Sirius Mining Merope',
       'Sirius Mining', 'Sirius Power']
      
fig = getCombinedFactionGraph( siriusFactions, titleSubject="Combined Sirius Factions")
fig.show()

In [28]:
csa = hist.getRawDataFrame()[['system','faction','updated','influence']]
display(csa)

,system,faction,updated,influence
0,Abroin,Abroin Universal PLC,2018-07-01,49.849500
1,Abroin,Abroin Universal PLC,2018-08-01,45.600000
2,Abroin,Abroin Universal PLC,2018-09-01,46.750567
3,Abroin,Abroin Universal PLC,2018-10-01,59.000000
4,Abroin,Abroin Universal PLC,2018-11-01,54.700000
...,...,...,...,...
17751,Pleiades Sector JC-V d2-62,Aegis Research,2020-10-12,1.393000
17752,California Sector JH-V c2-12,Turner Research Group,2020-10-12,100.000000
17753,California Sector BV-Y c7,Turner Research Group,2020-10-12,10.721400
17754,Delphi,Janus Incorporated,2020-10-12,9.625100


In [29]:
csa.columns

Index(['system', 'faction', 'updated', 'influence'], dtype='object')

In [30]:
#csa = csa.rename(columns={'system':'string', 'faction':'string', 'updated':'datetime', 'influence':'float', 'control':'boolean', 'region':'int','population':'float'})
csa = csa.rename(columns={'int':'id',
                          'string':'system',
                          'string':'faction',
                          'datetime': 'updated', 
                          'float':'influence',
                          'boolean':'control',
                          'int':'region',
                          'float': 'population'})

In [31]:
csa

,system,faction,updated,influence
0,Abroin,Abroin Universal PLC,2018-07-01,49.849500
1,Abroin,Abroin Universal PLC,2018-08-01,45.600000
2,Abroin,Abroin Universal PLC,2018-09-01,46.750567
3,Abroin,Abroin Universal PLC,2018-10-01,59.000000
4,Abroin,Abroin Universal PLC,2018-11-01,54.700000
...,...,...,...,...
17751,Pleiades Sector JC-V d2-62,Aegis Research,2020-10-12,1.393000
17752,California Sector JH-V c2-12,Turner Research Group,2020-10-12,100.000000
17753,California Sector BV-Y c7,Turner Research Group,2020-10-12,10.721400
17754,Delphi,Janus Incorporated,2020-10-12,9.625100


In [32]:
csa['system'].replace('', np.nan, inplace=True)
csa['faction'].replace('', np.nan, inplace=True)
csa['updated'].replace('', np.nan, inplace=True)
csa['influence'].replace('', np.nan, inplace=True)

In [33]:
pd.set_option('mode.chained_assignment', None)


In [34]:
from craid.eddb.loader.strategy.AWSLoader import LoadDataFromAWS
loader = LoadDataFromAWS(forceWebDownload=False, useSmol=False)
fName = loader.find_data_file('history.jsonl')
dataframe = pd.read_json(fName, lines=True, compression='infer')
dataframe = dataframe[ dataframe.faction != "Benton's Gang"]
dataframe = dataframe[ dataframe.faction != "Bentonian Party"]
dataframe = dataframe[ dataframe.faction != "Worster Insurance"]



Found data file: /tmp/history.jsonl.gz


In [35]:
csa.attrs

{}

In [36]:
dataframe = dataframe.rename(columns={'string':'system',
                          'string':'faction',
                          'datetime': 'updated', 
                          'float':'influence',
                          'boolean':'control',
                          'int':'region',
                          'float': 'population'})

In [39]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)

In [37]:
dataframe

,system,faction,updated,influence,control,region,population
0,Abroin,Abroin Universal PLC,1530403200000,49.849500,False,6,0
1,Abroin,Abroin Universal PLC,1533081600000,45.600000,False,6,0
2,Abroin,Abroin Universal PLC,1535760000000,46.750567,False,6,0
3,Abroin,Abroin Universal PLC,1538352000000,59.000000,False,6,0
4,Abroin,Abroin Universal PLC,1541030400000,54.700000,False,6,0
...,...,...,...,...,...,...,...
17751,Pleiades Sector JC-V d2-62,Aegis Research,1602457715000,1.393000,False,1,1200000
17752,California Sector JH-V c2-12,Turner Research Group,1602439649000,100.000000,True,3,10000
17753,California Sector BV-Y c7,Turner Research Group,1602419720000,10.721400,False,3,5000
17754,Delphi,Janus Incorporated,1602454904000,9.625100,False,1,300000


In [55]:
getFactionInfluenceGraph("Sirius Mining Merope").show()

In [56]:
getFactionInfluenceGraph("Sirius Hyperspace").show()

In [44]:
getFactionInfluenceGraph("Pleiades Resource Enterprise").show()

In [47]:
df_fac = dataframe.faction.unique().tolist()

def Diff(li1, li2): 
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
    return li_dif 

Diff(df_fac, badGuys)

In [57]:
#tf = csa[aegis & october]
tf = dataframe
tf = tf.reset_index()
tf = tf.sort_values( ['system','updated'])
tf = tf.rename(columns={'string':'system',
                          'string':'faction',
                          'timestamp': 'updated', 
                          'float':'influence',
                          'boolean':'control',
                          'int':'region',
                          'float': 'population'})
tf['updated'] = pd.to_datetime(tf.updated, unit='ms')
tf['updated'] = tf.updated.dt.round('D')
tf = tf.drop_duplicates(subset=['system', 'faction', 'updated'])
target = tf

mf = tf
#mf['updated'] = tf.updated.dt.round('MS')
mf['updated'] = pd.to_datetime(mf.updated).dt.to_period('M').dt.to_timestamp()
mf = mf.drop_duplicates(subset=['system', 'faction', 'updated'])
hurm = mf.groupby(['updated','faction']).count().reset_index()
hurm

In [51]:
def getTestGraph( faction: str):
  labels = {'x':'Day', 'y':"Total Systems"}
  foo = hurm[hurm['faction']== faction]
  fig = go.Figure(data=go.Scatter(x=foo.updated, y=foo.system, mode='lines+markers'))

  #fig = go.Figure()
  #factions = hurm['faction'].unique().tolist()
  #for fac in factions:
  #  foo = hurm[hurm['faction']== fac]
  #  stage3 = foo[['system','faction','updated']].groupby(['updated']).count().reset_index()
  #  fig.add_trace(go.Scatter(x=stage3['updated'], y=stage3['count'],
  #                  mode='lines',
  #                  stackgroup="b",
  #                  name=fac,
  #                  stackgaps="interpolate"))

  #if titleSubject is None:
  #  titleSubject = "+".join(factions)

  title = f"System Presences of {faction}"
  fig.update_layout(title=title,
                    xaxis_title="Date",
                    yaxis_title="Total Systems")
  #getScroller(fig)
  return fig

In [52]:
getTestGraph("Pleiades Resource Enterprise")

In [53]:
getTestGraph("Sirius Mining Merope")

In [54]:
getTestGraph("Sirius Hyperspace")